In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import helper_mate as h
import helper as hd
from colorscheme import *

from gt_learner import GT_learner

In [ ]:
# Data parameters
BLOCK_SIZE = 2
N_BATCHES = 4
ALPHA_LIST = [0,90]
N_RUNS = 100

# Agent parameters
SIGMA_R = 200.0
PP_THRESHOLD = 0.4
D = 10
#N_PARTICLES = 256

In [ ]:
# Generate N_RUNS datasets
datasets = [h.generate_batch_data(ALPHA_LIST, BLOCK_SIZE, N_BATCHES) for i in range(N_RUNS)]

# Define models to be tested
model_set = ['x', 'y', '1x2D', '2x1D_bg']

# Compute mllhs
results = {'gt': []}
pbar = tf.keras.utils.Progbar(N_RUNS)
for data in datasets:
    result = {}
    result['gt'] = GT_learner(data, SIGMA_R, model_set)
    results['gt'].append(result['gt'])
    pbar.add(1)

In [ ]:
# Compute switching times
switch_times = {}
for agent in ['gt']:
    switch_times[agent] = [hd.model_change_time(results[agent][i], model_set[-1])
                                 for i in range(N_RUNS)]

In [ ]:
#above two plots side by side in subfigure
fig, ax = plt.subplots(1,2)
ax[0].hist(switch_times['gt'], bins = 20)
ax[1].hist(switch_times['gt'], bins = 20, cumulative = True)
plt.gcf().set_size_inches((8,3))
#suptitle including SIGMA_R
plt.suptitle('Switching time distribution for GT learner, $\sigma_r$ = ' + str(SIGMA_R), fontsize = 14)
plt.tight_layout()
plt.subplots_adjust(top=0.85)
ax[0].set_xlabel('Switching time')
ax[0].set_ylabel('Occurences/100 runs')
plt.show()

In [ ]:
# Plot mllhs
i = 1
hd.plot_mmllh_curves(results['gt'][i], model_set, T=len(data['c']), color_dict=model_colors_gergo, figsize=(15,4),data=datasets[i])
print(switch_times['gt'][i])

In [ ]:
results['gt'][i]['prominent_models']

In [ ]:
transitions = []
# iterate over each pair of models from the above list
for i in range(len(results['gt'][i]['prominent_models'])-1):
    transitions.append([results['gt'][i]['prominent_models'][i],results['gt'][i]['prominent_models'][i+1]])
transitions
# tally unique elements in transitions
from collections import Counter
Counter([tuple(x) for x in transitions])


In [ ]:
results['gt'][i]['prominent_models'][i]

In [ ]:
# plot a directed graph with networkx
import networkx as nx
G = nx.DiGraph()
G.add_edge('x', 'y', weight=0)
G.add_edge('x', '1x2D', weight=0.2)
G.add_edge('x', '2x1D_bg', weight=0.2)
G.add_edge('y', '1x2D', weight=0.5)
G.add_edge('y', '2x1D_bg', weight=0.5)
G.add_edge('1x2D', '2x1D_bg', weight=1)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)
labels = nx.get_edge_attributes(G,'weight')
nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
plt.show()


